## Intro

# How many Restless are there in the Necropolis of Warsaw?
## Part 1. Date of Death

Actually, there are two questions:

**AoD)** What is the **_age of death_** of how many wraiths across the population of Necropolis? AoD alters demeanor & nature of wraiths' psyche & shadow.

**DoD)** What is the **_date of death_** of how many wraiths? DoD defines culture context of a character and certain aspects of his/her social position.

Both questions share the same demographic model of getting to the Shadowlands, migrations & depopulation (Ascension/Oblivion/decorpsing) and differs only in entry data. In this part of the article, let's take **DoD** on the workshop and try to develop a prototypical model of it.

## Input data

Sources:
1. [Demography of Warsaw (Wikipedia)](
https://pl.wikipedia.org/wiki/Ludno%C5%9B%C4%87_Warszawy) 
2. [Historical demography of Poland (academic pdf)](
http://mbc.cyfrowemazowsze.pl/Content/18436/Historia%20Polski%20w%20Liczbach_1994%20198147.pdf)

In [5]:
from builtins import list, open, zip
from operator import itemgetter
from pprint import pprint
import numpy as np
import yaml


with open('how_many_wraiths_p1.yml') as f:
    data = yaml.load(f)

alive_population_table = sorted(data['alive_population'].items(), key=itemgetter(0))
alive_population_datapoints = list(zip(*alive_population_table))
START_YEAR = 1600  # alive_population_table[0][0]
END_YEAR = 1800  # alive_population_table[-1][0]


def year_range(end_year=END_YEAR):
    return range(START_YEAR, end_year + 1)


def alive_population(t):
    """(Alive) population of Warsaw, based on demographical data ([1], [2])"""
    return int(np.interp(t, *alive_population_datapoints))


def mortality_rate(t):
    """Very simple approach to estimate mortality"""
    return 0.01  # if is_peaceful(t) else 1

## Model

In [10]:
from builtins import int, range, sorted, sum
from functools import lru_cache


@lru_cache(maxsize=None)
def population_per_dod(t, dod):
    if t < START_YEAR:
        return 0
    if t == dod:
        return int(natural_growth(t) * enfant_survavibility_factor(t))
    return int(population_per_dod(t - 1, dod) * (1 - pass_away_factor(t) + migration_factor(t)))


def pass_away_factor(t):
    return 0


def migration_factor(t):
    return 0


def population(t):
    return sum(population_per_dod(t, dod) for dod in range(START_YEAR, t + 1))


def natural_growth(t):
    return int(alive_population(t) * mortality_rate(t) * wraith_turn_factor(t))


def wraith_turn_factor(t):
    return 0.8
    

def enfant_survavibility_factor(t):
    rate = 1 - enfant_oblivion_rate(t) - enfant_decorpsing_rate(t)
    return rate if rate > 0 else 0


def enfant_oblivion_rate(t):
    return 0.1


def enfant_decorpsing_rate(t):
    return 0.1


def pass_away_rate(t):
    rate = senior_oblivion_rate(t) + senior_ascension_rate(t) + senior_decorpsing_rate(t)
    return rate if rate < 1 else 1


def senior_oblivion_rate(t):
    return 0.02


def senior_ascension_rate(t):
    return 0.01


def senior_decorpsing_rate(t):
    return 0.01

## Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

population_full_values = {(t, dod): population_per_dod(t, dod) for t in year_range() for dod in year_range(t)}
population_datapoints = (list(year_range()), list(population_per_dod(END_YEAR, dod) for dod in year_range()))

plt.plot(*alive_population_datapoints, 'r-', *population_datapoints, 'b-')
plt.show()